In [1]:
# 프로젝트 디렉토리에서 실행
# jupyter notebook에서 django shell을 사용할 수 있도록 설정

import os
os.environ['DJANGO_SETTINGS_MODULE'] = 'config.settings'
os.environ['DJANGO_ALLOW_ASYNC_UNSAFE'] = 'true'

import django
django.setup()

In [2]:
from polls.models import Question, Choice

# 전체 조회
- all

In [3]:
# Model클래스이름.objects.xxxxxxx
qs = Question.objects.all()  #QuerySet
qs

<QuerySet [<Question: 좋아하는 동물은?>]>

In [4]:
# QuerySet.query: sql문 조회
print(qs.query)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question"


In [5]:
for q in qs:
    print(q, type(q))

좋아하는 동물은? <class 'polls.models.Question'>


In [6]:
# 특정 조회결과 하나 조회 - indexing
# 범위 조회: slicing
qs[0]

<Question: 좋아하는 동물은?>

In [7]:
qs[-1]  #음수 index는 사용X

AssertionError: Negative indexing is not supported.

In [8]:
qs.first()

<Question: 좋아하는 동물은?>

In [9]:
qs.last()

<Question: 좋아하는 동물은?>

In [10]:
qs = Choice.objects.all()
qs

<QuerySet [<Choice: 강아지>, <Choice: 고양이>]>

In [11]:
qs.order_by('choice_text')  #ASC

<QuerySet [<Choice: 강아지>, <Choice: 고양이>]>

In [12]:
qs.order_by('-choice_text')  #DESC

<QuerySet [<Choice: 고양이>, <Choice: 강아지>]>

In [13]:
print(qs.query)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice"


In [14]:
c = qs.first()
print(type(c))

<class 'polls.models.Choice'>


In [15]:
# 조회한 Model의 속성을 조회 => 특정 컬럼값 조회 => . 표기법
# Choice: id(pk), choice_text, vote, question_id
print(c.id, c.choice_text, c.vote, c.question_id)

1 강아지 0 1


In [19]:
for c in qs:
    print(c.id, c.choice_text, c.vote, c.question_id, sep='\t')

1	강아지	0	1
2	고양이	0	1


# where 절
- filter(조회 조건)
    - 조회 조건을 만족하는 0개 이상의 행을 조회
- exclude(조회 조건)
    - filter의 반대. 조회 조건을 만족하지 않는 0개 이상의 행을 조회(not)
- get(조회 조건)
    - 조회 조건을 만족하는 1개 행을 조회
- filter/exclude: QuerySet으로 반환
- get: Model 객체 반환

In [23]:
# where id = 1
# qs = Question.objects.filter(id=2)
# primary key 컬럼 조회: 컬럼명, pk
qs = Question.objects.filter(pk=1)
qs

<QuerySet [<Question: 좋아하는 동물은?>]>

In [24]:
qs.first().pub_date

datetime.datetime(2021, 5, 25, 5, 21, 54, 599953, tzinfo=<UTC>)

In [26]:
try:
    r = Question.objects.get(pk=2)
    print(r.pub_date)
except:
    print("조회 결과가 없습니다.")

조회 결과가 없습니다.


In [27]:
# 조회조건
# qs = Choice.objects.filter(id__gt=5)  # id > 5
# qs = Choice.objects.filter(id__gte=5)  # id >= 5
# qs = Choice.objects.filter(id__lt=5)  # id < 5
qs = Choice.objects.filter(id__lte=5)  #id <= 5
for q in qs:
    print(q.id, q.choice_text)


1 강아지
2 고양이


In [31]:
# qs = Question.objects.filter(question_text__startswith='가고')  #q_t like 가고%
# qs = Question.objects.filter(question_text__endswith='?')
qs = Question.objects.filter(question_text__contains='좋아')
print(qs.query)
for q in qs:
    print(q.id, q.question_text)

SELECT "polls_question"."id", "polls_question"."question_text", "polls_question"."pub_date" FROM "polls_question" WHERE "polls_question"."question_text" LIKE %좋아% ESCAPE '\'
1 좋아하는 동물은?


In [34]:
# qs = Choice.objects.filter(id__in[2, 6, 7, 20])  #where id in [2, 6, 7, 20]
qs = Choice.objects.filter(id__range=[3, 7])  #between 3 and 7
print(qs.query)
for q in qs:
    print(q.id, q.choice_text)

SELECT "polls_choice"."id", "polls_choice"."choice_text", "polls_choice"."vote", "polls_choice"."question_id" FROM "polls_choice" WHERE "polls_choice"."id" BETWEEN 3 AND 7


In [35]:
qs = Choice.objects.filter(pk=3, choice_text='')
qs

<QuerySet []>

# 컬럼 선택
select 컬럼, 컬럼
- values(field이름, ...)

In [37]:
qs = Choice.objects.all().values('choice_text', 'vote')
print(qs.query)
for q in qs:
    print(type(q))

SELECT "polls_choice"."choice_text", "polls_choice"."vote" FROM "polls_choice"
<class 'dict'>
<class 'dict'>
